<a href="https://colab.research.google.com/github/ohmreborn/question-generation-AIB2023/blob/main/preprocess_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
with open('/content/drive/MyDrive/unified_abstract_infill.jsonl') as f:
  x = [json.loads(i)['text'] for i in list(f) ]

In [ ]:
import pandas as pd
df = pd.DataFrame(x,columns=['text'])

In [ ]:
df = df[df['text'].str.startswith('Background:')] 
df = df.reset_index(drop=True)
df

In [ ]:
df.to_csv('dataset.csv',index=False)

In [ ]:
df

In [ ]:
preprocess(df.iloc[:10])

In [ ]:
import numpy as np
import re

In [ ]:

def setup(text):
  conversation = text.split('<human>:',1)
  conversation[1] = '<human>:' + conversation[1]
  # conversation[0] = conversation[0][11:]
  return conversation

In [ ]:
import re
def clean(x):
  x = re.sub('😃🧘😂🌍🍞🚗📞🎉❤🍆👨👩👧','',x)
  x = re.sub('http.+?', '', x)
  # x = re.sub(r'[^\w\s]','',x)
  return x

In [ ]:
def preprocess(df):
  df.loc[:,['Background:','<human>:_<bot>:']] = np.stack(df['text'].apply(lambda x:setup(x)).values)

  df['Background:'] = df['Background:'].apply(lambda x:clean(x))
  df['<human>:_<bot>:'] = df['<human>:_<bot>:'].apply(lambda x:clean(x))
  return df

In [ ]:
import json
def save_file(start,stop,df):
  with open('output.jsonl', 'a') as outfile:
      for i in range(start,stop):
        data = df.loc[i].to_dict()
        json.dump(data, outfile)
        outfile.write('\n')
  return stop

In [ ]:
from tqdm import tqdm
start = 0
with tqdm() as bar:
  for i,df in enumerate(pd.read_csv('dataset.csv',chunksize=1_000)):
    length = df.shape[0]
    df = preprocess(df)
    save_file(start,start+length,df)
    start += length
    bar.update()

In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': f'output.jsonl',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/content/output.jsonl', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()